In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import json
from datetime import datetime
import os

In [ ]:
from datetime import datetime

def convert_time_to_milliseconds(event_time, start_time):
    time_part = event_time.split()[-2] + event_time.split()[-1].replace(' :', ':').replace(': ', ':')
    event_time_obj = datetime.strptime(time_part, "%H:%M:%S.%f")
    return (event_time_obj - start_time).total_seconds() * 1000


def label_impasse(time_col, object_finish_data, exam_data, start_time):
    labels = ["Unknown"] * len(time_col)
    print(object_finish_data)

    for obj in object_finish_data:
        print(obj)
        if obj["SectionNum"] == "1":
            finish_click_time = obj["finishClick"]
            print(finish_click_time)
            event_time_milliseconds = convert_time_to_milliseconds(finish_click_time, start_time)


            nearest_start_idx = nearest_multiple_of_8(event_time_milliseconds - 5000)
            nearest_end_idx = nearest_start_idx + 125

            nearest_start_idx = max(nearest_start_idx, 0)
            nearest_end_idx = max(nearest_end_idx, 0)


            for idx in range(nearest_start_idx, nearest_end_idx):
                labels[idx] = "Impasse"


    for section in exam_data["SectionList"]:
        for obj in section["ObjectList"]:
            for tag_event in obj["tagDetagList"]:
                for timestamp in tag_event["timestamps"]:
                    event_time_milliseconds = convert_time_to_milliseconds(timestamp["time"], start_time)
                    closest_idx = event_time_milliseconds


                    nearest_start_idx = nearest_multiple_of_8(closest_idx - 5000)
                    nearest_end_idx = nearest_start_idx + 125


                    nearest_start_idx = max(nearest_start_idx, 0)
                    nearest_end_idx = max(nearest_end_idx, 0)


                    if timestamp["name"] == "TagHandMenuPumpTime":
                        for idx in range(nearest_start_idx, nearest_end_idx):
                            labels[idx] = "Aha"

                        walking_start_click_idx = nearest_multiple_of_8(closest_idx - 7000)
                        walking_end_click_idx = walking_start_click_idx + 125
                        for idx in range(walking_start_click_idx, walking_end_click_idx):
                              labels[idx] = "Walking"


                        if event_time_milliseconds < max(time_col):
                            doing_other_tasks_start_idx = nearest_multiple_of_8((closest_idx))
                            doing_other_tasks_end_idx = doing_other_tasks_start_idx + 125
                            for idx in range(doing_other_tasks_start_idx, doing_other_tasks_end_idx):
                              labels[idx] = "Doing Other Task"

                    elif timestamp["name"] == "DetagHandMenuPumpTime":
                      for idx in range(nearest_start_idx, nearest_end_idx):
                        labels[idx] = "Re-evaluation"
                    elif timestamp["name"] == "BTButtonPressTime":
                        for idx in range(nearest_start_idx, nearest_end_idx):
                          labels[idx] = "Impasse"
    return labels

def nearest_multiple_of_8(idx):
    return round(idx / 8) - 1



In [ ]:

base_dir = '/content/drive/MyDrive/Project/'
cleaned_data_dir = os.path.join(base_dir, 'ICA')
exam_json_dir = os.path.join(base_dir, 'ExaminatioJson_Files')
finish_json_dir = os.path.join(base_dir, 'FinishJson_Files')
labeled_data_dir = os.path.join(base_dir, 'DataLabeled_1s_set4')

os.makedirs(labeled_data_dir, exist_ok=True)

In [ ]:
for filename in os.listdir(cleaned_data_dir):
    if filename.startswith('Subject_'):
        subject_number = filename.split('_')[1]
        print(f"Processing Subject {subject_number}")

        try:

            df = pd.read_csv(os.path.join(cleaned_data_dir, filename), delimiter='\t')

            if df.shape[1] < 17:
                print(f"Error: Expected at least 17 columns in {filename}, found {df.shape[1]}. Skipping...")
                continue

            time = df.iloc[:, 0]
            channels = df.iloc[:, 1:17]

            try:
                with open(os.path.join(exam_json_dir, f'Subject_{subject_number}_ExaminatioJson.txt'), 'r') as file:
                    exam_data = json.load(file)
                with open(os.path.join(finish_json_dir, f'Subject_{subject_number}_FinishJson.txt'), 'r') as file:
                    finish_data = json.load(file)
                    object_finish_data = finish_data.get("Objectfinish")
            except FileNotFoundError as e:
                print(f"File not found: {e}. Skipping subject {subject_number}.")
                continue
            except json.JSONDecodeError:
                print(f"Error decoding JSON for Subject {subject_number}. Skipping...")
                continue


            start_time_str = finish_data.get("StartTime")
            if not start_time_str:
                print(f"Start time not found for Subject {subject_number}. Skipping...")
                continue

            start_time = datetime.strptime(start_time_str, "%H:%M:%S")
            print("Start Time:", start_time)

            labels = label_impasse(time, object_finish_data, exam_data, start_time)


            df['Labels'] = labels
            # print(df.head())


            # filtered_df = df[df['Labels'] == 'Aha']
            # print(f"Filtered data shape for Subject {subject_number}: {filtered_df.shape}")

            # df['Labels'] = df['Labels'].replace("Unknown", "Walking")
            # print(df.head())

            output_file = os.path.join(labeled_data_dir, f'Subject_{subject_number}_Labeled_set4.csv')
            df.to_csv(output_file, index=False)
            print(f"Labeled data for Subject {subject_number} saved successfully!")
            print(f"Final shape of data for Subject {subject_number}: {df.shape}")

        except Exception as e:
            print(f"An error occurred while processing Subject {subject_number}: {e}")

Processing Subject 49
Start Time: 1900-01-01 15:49:42
[{'SectionNum': '1', 'finishClick': '2024 - 07 - 12 15: 58:48.357', 'finishConfirm': '2024 - 07 - 12 15: 58:57.296', 'Selection': 'A'}, {'SectionNum': '2', 'finishClick': '2024 - 07 - 12 16: 03:26.498', 'finishConfirm': '2024 - 07 - 12 16: 03:31.198', 'Selection': 'A'}]
{'SectionNum': '1', 'finishClick': '2024 - 07 - 12 15: 58:48.357', 'finishConfirm': '2024 - 07 - 12 15: 58:57.296', 'Selection': 'A'}
2024 - 07 - 12 15: 58:48.357
{'SectionNum': '2', 'finishClick': '2024 - 07 - 12 16: 03:26.498', 'finishConfirm': '2024 - 07 - 12 16: 03:31.198', 'Selection': 'A'}
Labeled data for Subject 49 saved successfully!
Final shape of data for Subject 49: (174516, 18)
Processing Subject 29
Start Time: 1900-01-01 16:07:45
[{'SectionNum': '1', 'finishClick': '2024 - 04 - 19 16: 12:09.930', 'finishConfirm': '2024 - 04 - 19 16: 12:14.862', 'Selection': 'B'}, {'SectionNum': '2', 'finishClick': '2024 - 04 - 19 16: 16:14.098', 'finishConfirm': '2024 -

In [ ]:
def extract_segments(df):
    segments = []
    labels = []

    current_segment = []
    current_label = None

    for idx, row in df.iterrows():
        label = row['Labels']

        if label in ['Aha', 'Re-evaluation', 'Impasse', 'Doing Other Task', 'Walking'] and current_label is None:

            current_label = label
            current_segment = [row[1:17].values]
            print(f"Started segment for {label} at index {idx}")

        elif label == 'Unknown' and current_label is not None:

            segments.append(np.array(current_segment))
            labels.append(current_label)
            print(f"Ended segment for {current_label} at index {idx}, segment length: {len(current_segment)}")
            current_label = None
            current_segment = []

        elif current_label is not None:
            current_segment.append(row[1:17].values)

    return segments, labels

In [ ]:
import os
import pandas as pd
import numpy as np

labeled_data_dir = '/content/drive/MyDrive/Project/DataLabeled_1s_set4'

all_segments = []
all_labels = []

print("Loading data...")

file_list = [filename for filename in os.listdir(labeled_data_dir) if filename.startswith('Subject_') and filename.endswith('_Labeled_set4.csv')]
total_files = len(file_list)

print(f'Total files to process: {total_files}')

for index, filename in enumerate(file_list):
    print(f'Processing file {index + 1}/{total_files}: {filename}')
    df = pd.read_csv(os.path.join(labeled_data_dir, filename))

    segments, labels = extract_segments(df)
    all_segments.extend(segments)
    all_labels.extend(labels)

    files_left = total_files - (index + 1)
    print(f'Files left to process: {files_left}')

print(f'Total segments extracted: {len(all_segments)}')


Loading data...
Total files to process: 27
Processing file 1/27: Subject_49_Labeled_set4.csv
Started segment for Walking at index 33976
Ended segment for Walking at index 34101, segment length: 125
Started segment for Aha at index 34226
Ended segment for Aha at index 34351, segment length: 125
Started segment for Doing Other Task at index 34851
Ended segment for Doing Other Task at index 34976, segment length: 125
Started segment for Impasse at index 40070
Ended segment for Impasse at index 40195, segment length: 125
Started segment for Re-evaluation at index 41087
Ended segment for Re-evaluation at index 41212, segment length: 125
Started segment for Impasse at index 46190
Ended segment for Impasse at index 46315, segment length: 125
Started segment for Impasse at index 50927
Ended segment for Impasse at index 51052, segment length: 125
Started segment for Impasse at index 67669
Ended segment for Impasse at index 67794, segment length: 125
Started segment for Walking at index 81453
En

In [ ]:
len(all_labels)

937

In [ ]:
output_file_all = '/content/drive/MyDrive/Project/ExtractedSegments_1s_set4.npy'
all_segments_array = np.array(all_segments, dtype=object)
np.save(output_file_all, all_segments_array)
print(f'Total segments extracted: {len(all_segments)}')
print(f'Segments saved to: {output_file_all}')

Total segments extracted: 937
Segments saved to: /content/drive/MyDrive/Project/ExtractedSegments_1s_set4.npy


In [ ]:
output_file_all_labels = '/content/drive/MyDrive/Project/ExtractedLabelas_1s_set4.npy'
all_labels = np.array(all_labels, dtype=object)
np.save(output_file_all_labels, all_labels)
print(f'Total Labels extracted: {len(all_labels)}')
print(f'Segments saved to: {output_file_all_labels}')

Total Labels extracted: 937
Segments saved to: /content/drive/MyDrive/Project/ExtractedLabelas_1s_set4.npy
